In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset

block_size = 3    # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]   # crop and append
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])      #80%
Xdev, Ydev = build_dataset(words[n1:n2])  #10%
Xte, Yte = build_dataset(words[n2:])      #10%

In [5]:
Xtr.shape, Ytr.shape  # Dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [6]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 #the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)   # for reproducibility
C = torch.randn((vocab_size, n_embd),             generator = g)  
W1 = torch.randn((n_embd * block_size, n_hidden), generator = g) * (5/3) / ((n_embd * block_size)**0.5)
#b1 = torch.randn(n_hidden,                        generator = g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator = g) * 0.01
b2 = torch.randn(vocab_size,                      generator = g) * 0

#BatchNorm Parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias,]
print(sum(p.nelement() for p in parameters))   # number of parameters in total
for p in parameters:
    p.requires_grad = True

12097


In [7]:
max_steps = 200000
lossi = []
batch_size = 32

for i in range(max_steps):
    
    #minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]  #batch X, Y
    
    # Forward Pass
    emb = C[Xb]  # Embed characters into vectors
    embcat = emb.view(emb.shape[0], -1)  #Concatenate the vectors
    hpreact = embcat @ W1 #+ b1   #hidden layer pre-activation

    #BatchNorm layer
    #-------------------------------------------------------------
    bnmeani = hpreact.mean(0, keepdims= True)
    bnstdi = hpreact.std(0, keepdims= True)
    hpreact = bngain * ((hpreact - bnmeani) / bnstdi) + bnbias
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    #-------------------------------------------------------------

    #Non-Linearity
    h = torch.tanh(hpreact)  # hidden layer
    logits = h @ W2 + b2    # output layer
    loss = F.cross_entropy(logits, Yb)  #loss   
    
    # Backward Pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update the parameters
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #Trach stats
    if i%10000 == 0:  #print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 3.2760
  10000/ 200000: 2.2798
  20000/ 200000: 2.2627
  30000/ 200000: 2.1540
  40000/ 200000: 2.5208
  50000/ 200000: 2.1863
  60000/ 200000: 2.0329
  70000/ 200000: 2.3750
  80000/ 200000: 2.0455
  90000/ 200000: 2.0192
 100000/ 200000: 2.2307
 110000/ 200000: 2.1016
 120000/ 200000: 2.0444
 130000/ 200000: 2.2681
 140000/ 200000: 2.1495
 150000/ 200000: 1.8622
 160000/ 200000: 2.0257
 170000/ 200000: 1.5726
 180000/ 200000: 1.8812
 190000/ 200000: 1.9315


In [8]:
@torch.no_grad()  #this decorator disables the gradient tracking
def split_loss(split):
    x, y = {
        'test': (Xte, Yte),
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev)
    }[split]
    emb = C[x]  #(N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1)  #concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 #+ b1
    hpreact = bngain * ((hpreact - bnmean_running) / bnstd_running) + bnbias
    h = torch.tanh(hpreact)  #(N, n_hidden)
    logits = h @ W2 + b2  #(N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.065136194229126
val 2.105121612548828


In [9]:
# Sample from the model

g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(10):

    out = []
    context = [0] * block_size # Initialize with all ...
    while True:
        emb = C[torch.tensor([context])]  # (1, block_size, d)
        embcat = emb.view(emb.shape[0], -1)  #concat into (N, block_size * n_embd)
        hpreact = embcat @ W1 #+ b1
        hpreact = bngain * ((hpreact - bnmean_running) / bnstd_running) + bnbias
        h = torch.tanh(hpreact)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim = 1)
        ix = torch.multinomial(probs, num_samples = 1, generator = g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

mora.
mayah.
see.
med.
ryla.
reisha.
ejdrie.
caileed.
eliah.
milopileigh.
estanara.
mykimhone.
cayshabergihiriel.
kindreelynn.
novana.
ubented.
ryylani.
els.
kayshayton.
mahil.
